In [1]:
from fasttext_comparison_function import eval_dataset

In [7]:
import fasttext
import os
import sys
import os.path
import numpy as np
from numpy import array
from sklearn.model_selection import KFold
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [63]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
df = pd.read_csv(data_in_path)
df = df.dropna()
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
# df['data'] = df['labels'] + " " + df['descriptions']
df.head()


,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [64]:
data_path = r"..\dataset\ft_bal.txt"
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
df['data'] = df['labels'] + " " + df['descriptions']
df_file = df[['data']]
df_file.to_csv(data_path,index=False,header=False,sep="\t",encoding='utf-8-sig')
df_file.shape

(29625, 1)

# testing initial model

In [83]:

# path_train = os.path.dirname(__file__) + './tmp/tmp_train.txt'
# path_test = os.path.dirname(__file__) + './tmp/tmp_test.txt'

path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\output_ft_regular.txt"
data_set = data_path
f_out = data_out_path
# try:
print("Converting dataset to array")
# f = open(data_set, 'r+', encoding="UTF-8")

data = array(df_file['data'])
# f.close()

# array for details
fold_outputs = []

# 10 fold loop
kfold = KFold(10, shuffle=True, random_state=1)
fold = 1
for train, test in kfold.split(data):
    print("New tenfold iteration:", str(fold), "-----------------------------------------")
    print("Creating train file")
    tmp_train = data[train]
    pd.DataFrame(tmp_train).to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    print("Creating test file")
    tmp_test = data[test]
    pd.DataFrame(tmp_test).to_csv(path_test,index=False,header=False,encoding='utf-8-sig')



    print("start training...")
    # use this model if you want to enable auto-tuning. Sadly we did not get it to work during our project
    # model = fasttext.train_supervised(input=path_train, autotuneValidationFile=path_test, autotuneDuration=600)
    # model = fasttext.train_supervised(input=path_train)  # normal fasttext classification without autotuning
    model = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)
    print("start testing...")
    res = model.test(path_test)
    # get benchmarks
    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))
    result = {
        '10-Fold iteration:': fold,
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }
    # log
    print("Fold over, here are results: ")
    print(json.dumps(result, indent=4))
    fold_outputs.append(result)
    fold += 1
print("Done with 10 fold validation")
# calculate over-all results
mean_recall = 0
mean_precision = 0
mean_f1 = 0
for f in fold_outputs:
    mean_f1 += (f['F1'] / 10)
    mean_recall += (f['Recall'] / 10)
    mean_precision += (f['Precision'] / 10)
    # compile results as json
output = {
    'Results': {
        'F1': mean_f1,
        'Recall': mean_recall,
        'Precision': mean_precision
    },
    'Details': fold_outputs
}
dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")



Converting dataset to array
New tenfold iteration: 1 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
Fold over, here are results: 
{
    "10-Fold iteration:": 1,
    "F1": 0.6282642089093702,
    "Recall": 0.6282642089093702,
    "Precision": 0.6282642089093702
}
New tenfold iteration: 2 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
Fold over, here are results: 
{
    "10-Fold iteration:": 2,
    "F1": 0.6763803680981595,
    "Recall": 0.6763803680981595,
    "Precision": 0.6763803680981595
}
New tenfold iteration: 3 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
Fold over, here are results: 
{
    "10-Fold iteration:": 3,
    "F1": 0.6437308868501529,
    "Recall": 0.6437308868501529,
    "Precision": 0.6437308868501529
}
New tenfold iteration: 4 -----------------------------------

Plan / work steps
need to split the repo level data into test/train datasets

need to train a model with a simple test/ train split from the balanced data
then use that model to predict on each of the repo datasets

then split each repo dataset into test train

include the repo data inside the training data
then repredict with it
see if it improves it at all. 



Need to organize the data into separate groups for each of the different repo
then run the eval function over each of them. 



# Separating creating model and using it to predict repo issues

In [ ]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
data_df = pd.read_csv(data_in_path)
data_df = data_df.dropna()
data_df.head()

In [ ]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(data_df['descriptions'], data_df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

In [ ]:
df_train['data'] = df_train['labels'] + " " + df_train['descriptions']
df_train = df_train[['data']]

df_train['data'] = df_train['labels'] + " " + df_train['descriptions']
df_train = df_train[['data']]

In [ ]:
print("Creating train file")
pd.DataFrame(tmp_train).to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

print("Creating test file")
pd.DataFrame(tmp_test).to_csv(path_test,index=False,header=False,encoding='utf-8-sig')